# 구조화된 생성으로 근거 강조 표시가 있는 RAG 시스템 구축하기
_작성자: [Aymeric Roucher](https://huggingface.co/m-ric), 번역: [유용상](https://huggingface.co/4n3mone)_

**구조화된 생성**(Structured generation)은 LLM 출력이 특정 패턴을 따르도록 강제하는 방법입니다.

이 방법은 여러 가지 용도로 사용될 수 있습니다:
- ✅ 특정 키가 있는 딕셔너리 출력
- 📏 출력이 N글자 이상이 되도록 보장
- ⚙️ 더 일반적으로, 다운스트림 처리를 위해 출력이 특정 정규 표현식 패턴을 따르도록 강제
- 💡 검색 증강 생성(RAG)에서 답변을 뒷받침하는 소스를 강조 표시

이 노트북은 마지막 예시를 구체적으로 보여줍니다.

**➡️ 우리는 답변을 제공할 뿐만 아니라 이 답변의 근거가 되는 스니펫을 강조 표시하는 RAG 시스템을 구축합니다.**

_RAG에 대한 소개가 필요하다면, [이 쿡북](advanced_rag)을 확인해 보세요._

이 노트북은 먼저 프롬프트를 통한 구조화된 생성의 단순한 접근 방식을 보여주고 그 한계를 강조한 다음, 더 효율적인 구조화된 생성을 위한 제한된 디코딩(constrained decoding)을 시연합니다.

이 노트북은 HuggingFace Inference Endpoints를 활용합니다 (예제는 [서버리스](https://huggingface.co/docs/api-inference/quicktour) 엔드포인트를 사용하지만, [전용](https://huggingface.co/docs/inference-endpoints/en/guides/access) 엔드포인트로 변경할 수 있습니다), 또한 [outlines](https://github.com/outlines-dev/outlines)라는 구조화된 텍스트 생성 라이브러리를 사용한 로컬 추론 예제도 보여줍니다.

In [66]:
!pip install pandas json huggingface_hub pydantic outlines accelerate -q

In [67]:
import pandas as pd
import json
from huggingface_hub import InferenceClient

pd.set_option("display.max_colwidth", None)

In [68]:
repo_id = "mistralai/Mistral-Nemo-Instruct-2407"

llm_client = InferenceClient(model=repo_id, timeout=120)

# Test your LLM client
llm_client.text_generation(prompt="대한민국의 수도는?", max_new_tokens=50)

' 서울특별시입니다.'

## 모델에 프롬프트 제공하기

모델에서 구조화된 출력을 얻으려면, 충분히 성능이 좋은 모델에 적절한 지시사항을 포함한 프롬프트를 제공하면 됩니다. 대부분의 경우 이 방법이 잘 작동할 것입니다.

이번 경우, 우리는 RAG 모델이 답변뿐만 아니라 신뢰도 점수와 근거가 되는 스니펫도 함께 생성하기를 원합니다.

이러한 출력을 JSON 형식의 딕셔너리로 생성하면, 나중에 쉽게 처리할 수 있습니다 (여기서는 근거가 되는 스니펫을 강조하여 표시할 예정입니다).

In [69]:
RELEVANT_CONTEXT = """
문서:

오늘 서울의 날씨가 정말 좋네요.
Transformers에서 정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다.

"""

In [70]:
RAG_PROMPT_TEMPLATE_JSON= """문서를 기반으로 사용자 쿼리에 응답합니다.

다음은 문서입니다: {context}


답변을 JSON 형식으로 제공하고, 답변의 직접적 근거가 된 문서의 모든 관련 짧은 소스 스니펫과 신뢰도 점수를 0에서 1 사이의 부동 소수점으로 제공해야 합니다.
근거 스니펫은 전체 문장이 아닌 기껏해야 몇 단어 정도로 매우 짧아야 합니다! 그리고 문맥에서 정확히 동일한 문구와 철자를 사용하여 추출해야 합니다.

답변은 다음과 같이 작성해야 하며, “Answer:” 및 “End of answer.” 를 포함해야 합니다.

Answer:
{{
  “answer": 정답 문장,
  “confidence_score": 신뢰도 점수,
  “source_snippets": [“근거_1”, “근거_2”, ...]
}}
End of answer.

이제 시작하세요!
다음은 사용자 질문입니다: {user_query}.
Answer:
"""

In [71]:
USER_QUERY = "Transformers에서 정지 시퀀스를 어떻게 정의하나요?"

In [72]:
prompt = RAG_PROMPT_TEMPLATE_JSON.format(
    context=RELEVANT_CONTEXT, user_query=USER_QUERY
)
print(prompt)

문서를 기반으로 사용자 쿼리에 응답합니다.

다음은 문서입니다: 
문서:

오늘 서울의 날씨가 정말 좋네요.
Transformers에서 정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다.




답변을 JSON 형식으로 제공하고, 답변의 직접적 근거가 된 문서의 모든 관련 짧은 소스 스니펫과 신뢰도 점수를 0에서 1 사이의 부동 소수점으로 제공해야 합니다.
근거 스니펫은 전체 문장이 아닌 기껏해야 몇 단어 정도로 매우 짧아야 합니다! 그리고 문맥에서 정확히 동일한 문구와 철자를 사용하여 추출해야 합니다.

답변은 다음과 같이 작성해야 하며, “Answer:” 및 “End of answer.” 를 포함해야 합니다.

Answer:
{
  “answer": 정답 문장,
  “confidence_score": 신뢰도 점수,
  “source_snippets": [“근거_1”, “근거_2”, ...]
}
End of answer.

이제 시작하세요!
다음은 사용자 질문입니다: Transformers에서 정지 시퀀스를 어떻게 정의하나요?.
Answer:



In [73]:
answer = llm_client.text_generation(
    prompt,
    max_new_tokens=256,
)

answer = answer.split("End of answer.")[0]
print(answer)

{
  "answer": "Transformers에서 정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다.",
  "confidence_score": 0.95,
  "source_snippets": ["정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다."]
}



LLM의 출력은 딕셔너리의 문자열 표현입니다. 따라서 `literal_eval`을 사용하여 이를 딕셔너리로 로드합시다.

In [74]:
from ast import literal_eval

parsed_answer = literal_eval(answer)

In [75]:
def highlight(s):
    return "\x1b[1;32m" + s + "\x1b[0m"


def print_results(answer, source_text, highlight_snippets):
    print("Answer:", highlight(answer))
    print("\n\n", "=" * 10 + " Source documents " + "=" * 10)
    for snippet in highlight_snippets:
        source_text = source_text.replace(snippet.strip(), highlight(snippet.strip()))
    print(source_text)


print_results(
    parsed_answer["answer"], RELEVANT_CONTEXT, parsed_answer["source_snippets"]
)

Answer: Transformers에서 정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다.


 ========== Source documents ==========

문서:

오늘 서울의 날씨가 정말 좋네요.
Transformers에서 정지 시퀀스를 정의하려면 파이프라인 또는 모델에 stop_sequence 인수를 전달해야 합니다.




잘 작동합니다! 🥳

하지만 성능이 낮은 모델을 사용하는 경우는 어떨까요?

성능이 떨어지는 모델의 불안정한 출력을 시뮬레이션하기 위해, temperature 값을 높여보겠습니다.